In [3]:
import pandas
import matplotlib.pyplot as plt
import numpy as np

def scale(df, column_list): #-1 to 1
  for x in column_list:
    scaled = np.log(np.sqrt(df[x] - df[x].min() + 1))
    max_value = scaled.max()
    min_value = scaled.min()
    scaled = 2 * ((scaled - min_value) / (max_value - min_value)) - 1 # -1 to 1
    df[x] = scaled
  return df

def getStationary(df, fn, scaling=True):
  df['delta'] = (df['close'] / df ['open']) - 1
  df['amplitude'] = (df['high'] / df['low']) - 1
  df['close_over_low'] = (df['close'] / df ['low']) - 1
  del df['open']
  del df['high']
  del df['low']
  df['volume'].replace(0, 0.00001, inplace=True)

  # Scaling
  if scaling:
    df = scale(df, ['delta','amplitude','close_over_low','volume'])

  # Saving
  df.to_csv(f'../data/{fn}-full.csv')
  df.iloc[:500].to_csv(f'../data/{fn}-short.csv')
  df.iloc[:10000].to_csv(f'../data/{fn}-long.csv')
  return df

def clear():
  import os
  import re
  directory = '../data/'
  files = os.listdir(directory)
  files_to_delete = [file for file in files if re.search(r'-', file)]
  for file_to_delete in files_to_delete:
    os.remove(os.path.join(directory, file_to_delete))

def print_stats(df):
  for column in df.columns:
    min_value = df[column].min()
    max_value = df[column].max()
    avg_value = df[column].mean()
    has_nan = df[column].isna().any()
    print(f"Column '{column}': Min = {min_value}, Max = {max_value}, Mean = {avg_value}", {has_nan})


In [4]:
fn = 'bnb22'
dat = ((pandas.read_csv(f'../data/{fn}.csv',sep=',')).sort_values('date')[['date','open','high','low','close','volume']]).set_index('date')

try:
  dat_test = ((pandas.read_csv(f'../data/{fn}-full.csv',sep=',')).sort_values('date')[['date','close_over_low','delta','amplitude','volume']]).set_index('date')
except:
  pass

# clear()
getStationary(dat, fn, True)
# print_stats(dat_test)

,close,volume,delta,amplitude,close_over_low
date,,,,,
2022-01-01 00:00:00,512.3,-0.049193,-0.213218,-0.972629,-0.936372
2022-01-01 00:01:00,513.1,0.068005,-0.202542,-0.961726,-0.904670
2022-01-01 00:02:00,513.5,-0.075670,-0.224029,-0.978150,-0.936521
2022-01-01 00:03:00,513.1,0.301687,-0.266888,-0.969977,-0.984127
2022-01-01 00:04:00,512.8,-0.185005,-0.266898,-0.989073,-1.000000
...,...,...,...,...,...
2022-12-31 23:55:00,246.3,-0.150247,-0.239650,-0.994309,-0.983467
2022-12-31 23:56:00,246.3,-0.455261,-0.250816,-1.000000,-1.000000
2022-12-31 23:57:00,246.0,-0.048058,-0.284328,-0.982920,-1.000000
